# Tensorflow tutorial

In [ ]:
import tensorflow as tf

## Build and execute a simple graph

Tensorflow (and similar frameworks) work by building tensors and operations into a _graph_. The graph includes inputs (often implemented as `tf.placeholder` tensors, see later) and outputs. Importantly, we can use Tensorflow to perform automatic differentation of output loss tensors with respect to variables within the graph (used for gradient-based optimisation). 

Once the graph has been defined, it can then be executed inside a `tf.Session`. Tensorflow optimises the operations in the graph to ensure quick execution. Graph operations can also be placed on GPU hardware for extra speedup.

In [ ]:
x = tf.constant([1, 2])
y = tf.constant([4, 5])

z = tf.multiply(x, y)

print(z)

`z` is a `Tensor` object; it is the result of a graph operation (multiplication) on two other tensor objects, `x` and `y`. Printing `z` does not evaluate the graph operation, but simply returns the `Tensor` object.

To evaluate `z`, we need to run the graph with a `tf.Session()`.

In [ ]:
x = tf.constant([1, 2, 3])
y = tf.constant([4, 5, 6])

z = tf.multiply(x, y)

sess = tf.Session()

print(sess.run(z))

# We need to remember to close the session!
sess.close()

The `tf.Session()` is often instantiated with the python `with` statement. This ensures that the session is automatically closed when we are out of scope of of the `with` statement.

In [ ]:
x = tf.constant([1, 2, 3])
y = tf.constant([4, 5, 6])

print(x)

In [ ]:
output1 = tf.multiply(x, y)

with tf.Session() as sess:
    output = sess.run(output1)
    print(output)

## Creating trainable variables

Often we want to include variables in our graph. For example, these could be parameters of a machine learning model that we would like to fit during an optimisation process. 

In [ ]:
a = tf.Variable(2.0, name='a')
print(a)

In [ ]:
# If we run the following without the tf.cast operation then it will throw an error
output2 = tf.add(tf.cast(x, tf.float32), a)
print(output2)

We have defined a simple graph that adds `x` (a `tf.constant`) and `a` (a `tf.Variable`). If we now try to run this graph in a session, it will throw an error:

In [ ]:
with tf.Session() as sess:
    output = sess.run(output2)
    print(output)

Why? Because `a` is a variable, and that means that Tensorflow does not automatically have any value associated with that variable. As the error message says, `a` needs to be initialised before the graph can be run.

### Initializing variables

The following defines the initialiser operation that needs to be run before any variables can be used in graph executions. Recall that we specified a default value for `a` when we defined it.

In [ ]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(a))
    output = sess.run(output2)
    print(output)

Variables can also be passed random initialisers. Note that we can (and should!) also name our variables.

In [ ]:
b = tf.Variable(tf.random_normal([2, 2], stddev=0.1), name="b")
print(b)

Re-run the following a few times to randomly initialise the tensor `b`.

In [ ]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    output = sess.run(b)
    print(output)

In [ ]:
b

### `tf.get_variable`

Calling `tf.Variable` will always create a new variable. Running the following cell will create another tensor variable object, that replaces the old one stored in `b`.

In [ ]:
b = tf.Variable(tf.random_normal([2, 2], stddev=0.1), name="b")
print(b)

Compare the names of the variables above and note that Tensorflow automatically gave this last variable a unique name. The old `b` variable still exists in the graph! In fact, we can take a look at all the variables we have so far by running:

In [ ]:
tf.global_variables()

This poses a potential issue - sometimes we want to retrieve variables that we have already defined before. This is where `tf.get_variable()` comes in: using this will create a new variable if there isn't one already in the graph with the same name, or else will attempt to retrieve the variable that already exists.

In [ ]:
with tf.variable_scope('layer1'):
    b = tf.get_variable("b", initializer=tf.random_normal([2, 2], stddev=0.1))
    
print(b)

In the above we created another new variable, this time with `tf.get_variable()`. We also made use of the `tf.variable_scope()` to prepend the name with `layer1`: this is a useful habit to get into to organise your variables into name spaces.

Let's initialise and print out the value of this new variable as before:

In [ ]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    output = sess.run(b)
    print(output)

Take a look at the variables in our graph now:

In [ ]:
tf.global_variables()

Now say we would like to retrieve this last variable from the graph (named `layer1/b:0`). If we use `tf.get_variable()` as follows we will get an error:

In [ ]:
with tf.variable_scope('layer1'):
    b = tf.get_variable('b', shape=(2, 2), initializer=tf.random_normal_initializer())

This is a safety feature in Tensorflow - it forces us to be aware of when we are creating new variables, and when we are re-using existing variables.

If we know that we want to retrieve the variable, then we need to set reuse to `True` in the variable scope, as it says in the error message.

In [ ]:
with tf.variable_scope('layer1', reuse=True):
    b = tf.get_variable('b', shape=(2, 2), initializer=tf.random_normal_initializer())

In [ ]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    output = sess.run(b)
    print(output)

Final check that the previous cells didn't create any new variables:

In [ ]:
tf.global_variables()

A useful method (especially when working interactively, as in a notebook) to clear the current graph is `tf.reset_default_graph()`.

In [ ]:
tf.reset_default_graph()
tf.global_variables()

## Placeholders

Placeholders are another important type of tensor. We often use them to feed data into a model during the fitting process. That means that in order to execute a graph that depends on a placeholder tensor, we need to provide the value of that placeholder tensor.

Let's first create a placeholder:

In [ ]:
c = tf.placeholder(tf.float32, shape=(2,), name='input')
print(c)

Running the following cell will throw an error. It should be clear why - we are trying to run a graph (a very simple graph consisting only of the placeholder `c`) without providing a value for the placeholder. The error messageg explicitly tells us this.

In [ ]:
with tf.Session() as sess:
    output = sess.run(c)
    print(output)

In order to feed values to placeholders, we use the `feed_dict` keyword argument when running the graph in a session. This argument expects a dictionary, where relevant placeholder tensors are keys, and the values are the values those placeholders should take. These values often come from data in practice.

In [ ]:
import numpy as np

feed_dict = {c: np.array([3, 4])}

with tf.Session() as sess:
    output = sess.run(c, feed_dict=feed_dict)
    print(output)

When creating placeholders, it is possible to provide one or more of the shape dimensions as `None`. This means that the size of that dimension is only specified at graph execution time, i.e. when provided data to the graph through the `feed_dict`. 

For example, this is useful in cases where we would like to define a graph such that it processes multiple data points at once, but we would like to keep the actual number of data points flexible.

In [ ]:
d = tf.placeholder(tf.float32, shape=(None, 3), name='data_batch')

In [ ]:
data1 = np.random.randn(2, 3)

feed_dict = {d: data1}
with tf.Session() as sess:
    output = sess.run(d, feed_dict=feed_dict)
    print(output)

In [ ]:
data2 = np.random.randn(5, 3)

feed_dict = {d: data2}
with tf.Session() as sess:
    output = sess.run(d, feed_dict=feed_dict)
    print(output)

## More Tensorflow operations

Tensorflow comes with a large range of methods and operations that can be performed on tensors. Most things that we would want to do are available as tensorflow operations - check the docs to look for operations that you want to use.

In [ ]:
mat_inv = tf.matrix_inverse(b)
mat_vec_multiply = tf.matmul(mat_inv, tf.expand_dims(c, axis=1))
print(mat_vec_multiply)

In [ ]:
squeezed = tf.squeeze(mat_vec_multiply)
print(squeezed)

In [ ]:
feed_dict = {c: np.array([1, 1])}

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    output = sess.run([squeezed, mat_inv], feed_dict=feed_dict)
    print(output[0])
    print(output[1])